In [15]:
from pathlib import Path
import os
import pandas as pd
Path.cont = lambda x: list(os.scandir(x)) # scandir gives all content of a dir 
filedir = 'RIVM_data'
files = Path.cont(filedir)

def generate_df(filedir,filename,i):
    rename = {'Category':'Gemeente',
            'Aantal per 100.000 inwoners':'Aantal per 100000 inwoners'}
    drop = ['id','Indicator','Gemnr']
    if len(filename.name) > 30:
        df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',delimiter=';')
    else:
        df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',skiprows=1)
    for old,new in rename.items():
        if old in df.columns:
            df.rename(columns={old: new},inplace=True)
    for d in drop:
        if d in df.columns:
            df = df.drop([d], axis=1) 
    df['Datum'] = filename.name[:10]
    df['Count'] = str(i)

    return df
for i,filename in enumerate(files):
    print(filename.name)
    print(generate_df(filedir,filename,i).columns)
num_csv = len(files)
df = pd.concat([generate_df(filedir,filename,i) for i,filename in enumerate(files)])


df.head()


03-03-2020 20200308095848klik_corona03032020.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-04-2020 20200308095940isthisokklik_corona04032020.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-05-2020 20200308100010klik_corona05032020.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-06-2020 20200308100025klik_corona06032020_rec_0.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-07-2020 20200308031948klik_corona07032020.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-08-2020 20200308211244klik_corona08032020_rectificatie.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-09-2020 20200309161202klik_corona09032020_0.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-10-2020 20200312221603klik_corona10032020_2.csv
Index(['Gemeente', 'Aantal', 'Datum', 'Count'], dtype='object')
03-11-2020 20200312221524klik_corona11032020rectificatie.csv
Inde

,Aantal,Aantal per 100000 inwoners,BevAant,Count,Datum,Gemeente
0,0.0,0.0,0.0,0,03-03-2020,Aa en Hunze
1,0.0,0.0,0.0,0,03-03-2020,Aalsmeer
2,0.0,0.0,0.0,0,03-03-2020,Aalten
3,0.0,0.0,0.0,0,03-03-2020,Achtkarspelen
4,0.0,0.0,0.0,0,03-03-2020,Alblasserdam


In [19]:
df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_aantal = df_aantal.fillna(0)
df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_relative = df_relative.fillna(0)
df_aantal.head()

Count,Gemeente,0,1,10,11,12,13,14,15,16,17,2,3,4,5,6,7,8,9
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,'s-Gravenhage,0.0,0.0,8.0,9.0,10.0,14.0,17.0,21.0,27.0,36.0,0.0,2.0,2.0,2.0,2.0,3.0,7.0,7.0
2,'s-Hertogenbosch,0.0,0.0,11.0,14.0,14.0,20.0,24.0,30.0,35.0,38.0,0.0,0.0,2.0,2.0,3.0,4.0,5.0,8.0
3,* Bij 112 personen is de woonplaats niet van b...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,* postcode ontbreekt bij 37 pt en 1 woont in b...,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
1

1

In [23]:
import altair as alt
import pandas as pd
from vega_datasets import data
url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

columns = [str(day) for day in range(0, num_csv)]

slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend):
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns),
        default = '0'
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='redyellowgreen', type='symlog',domain=[200, 0])),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=350,
        height=400
    ).transform_filter(
        select_year
    )
    
    return chart
serve_chart(df_aantal, 'aantal') | serve_chart(df_relative, 'aantal')

alt.HConcatChart(...)

In [21]:
alt.hconcat(
    serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief')
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.HConcatChart(...)

In [65]:
df_aantal[df_aantal['Gemeente']=='Uden']

Count,Gemeente,0,1,2,3
296,Uden,53.0,60.0,60.0,71.0


In [66]:
df_relative[df_relative['Gemeente']=='Uden']

Count,Gemeente,0,1,2,3
296,Uden,126.8,143.6,143.6,169.9


In [37]:
df['Gemeente'].unique()

array(['Aa en Hunze', 'Aalsmeer', 'Aalten', 'Achtkarspelen',
       'Alblasserdam', 'Albrandswaard', 'Alkmaar', 'Almelo', 'Almere',
       'Alphen aan den Rijn', 'Alphen-Chaam', 'Altena', 'Ameland',
       'Amersfoort', 'Amstelveen', 'Amsterdam', 'Apeldoorn', 'Appingedam',
       'Arnhem', 'Assen', 'Asten', 'Baarle-Nassau', 'Baarn',
       'Barendrecht', 'Barneveld', 'Beek', 'Beekdaelen', 'Beemster',
       'Beesel', 'Berg en Dal', 'Bergeijk', 'Bergen (L)', 'Bergen (NH)',
       'Bergen op Zoom', 'Berkelland', 'Bernheze', 'Best', 'Beuningen',
       'Beverwijk', 'Bladel', 'Blaricum', 'Bloemendaal',
       'Bodegraven-Reeuwijk', 'Boekel', 'Borger-Odoorn', 'Borne',
       'Borsele', 'Boxmeer', 'Boxtel', 'Breda', 'Brielle', 'Bronckhorst',
       'Brummen', 'Brunssum', 'Bunnik', 'Bunschoten', 'Buren',
       'Capelle aan den IJssel', 'Castricum', 'Coevorden', 'Cranendonck',
       'Cuijk', 'Culemborg', 'Dalfsen', 'Dantumadiel', 'De Bilt',
       'De Fryske Marren', 'De Ronde Venen', 'De Wo

In [79]:
len(df_aantal.columns)


19